In [159]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup as Soup
import time
from random import randint
import csv

browser = webdriver.Chrome("chromedriver")
url = 'https://www.instagram.com/'
username = "ian.1031" #鍵入使用者名稱
password = "abclee1031" #鍵入密碼



# ------ 前往該網址 ------
browser.get(url) 

# ------ 填入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_elements_by_name('username')[0]
password_input = browser.find_elements_by_name('password')[0]

# ------ 輸入帳號密碼 ------
username_input.send_keys(username)
password_input.send_keys(password)


# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="loginForm"]/div/div[3]/button/div')))
login_click = browser.find_elements_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div')[0]
login_click.click()


# ------ 不儲存登入資料 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/main/div/div/div/div/button')))
store_click = browser.find_elements_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button')[0]
store_click.click()

# ------ 不開啟通知 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div/div/div[3]/button[2]')))                                                                                
notification_click = browser.find_elements_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]')[0]
notification_click.click()

# ------ 轉至主畫面 ------
time.sleep(randint(50,100)/100)
browser.get(url+username)

# ----- 儲存追蹤中帳戶 ----
following = []
time.sleep(randint(50,100)/100)
following_list = browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/a')
following_list.click()

time.sleep(randint(50,100)/100)
dialog = browser.find_element_by_xpath('/html/body/div[5]/div/div/div[2]')

#find number of followers
#allfoll=int(browser.find_element_by_xpath("//li[3]/a/span").text)
#scroll down the page

while True:
    page = browser.page_source
    elm = Soup(page,"lxml")
    lst = elm.find_all("a",{"class":"FPmhX notranslate _0imsa"})
    
    browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", dialog)
    time.sleep(randint(100,150)/100)
    
    page = browser.page_source
    elm = Soup(page,"lxml")
    next_lst = elm.find_all("a",{"class":"FPmhX notranslate _0imsa"})
    if lst == next_lst:
        break

tg_lst = []
for i in lst:
    tg_lst.append(i.text)
    



result = []

#random = [randint(0,len(tg_lst)) for i in range(20)]
for case in range(len(tg_lst)):
    browser.get(url+tg_lst[case])
    # ------ 貼文、追蹤、粉絲人數 ---
    num_info = []
    num = browser.find_elements_by_class_name('g47SY')
    for i in num:
        num_info.append(i.text)

    # ------ 自介 ---
    intro_info = []
    intro = browser.find_elements_by_class_name('-vDIg')
    for i in intro:
        intro_info.append(i.text)

    if (num_info[0] == 'NaN') or (int(num_info[0])>500):
        info = []
        info.append(num_info)
        info.append(intro_info)
        info.append([''])
        info.append([''])
        result.append(info)
        print(tg_lst[case] + ":總共取得0篇貼文連結")
        continue
    
    # ------ 儲存貼文連結 -----#######################################存不到url!!!!!!!!
    post_url = []
    while True:
        html = browser.page_source
        soup = Soup(html, 'lxml')
        img = soup.find_all("img",{"class":"FFVAD"})

        scroll = 'window.scrollTo(0, document.body.scrollHeight);'
        browser.execute_script(scroll)
        time.sleep(randint(350,400)/100) # 等待網頁加載#########
            
        html = browser.page_source
        soup = Soup(html, 'lxml')
        next_img = soup.find_all("img",{"class":"FFVAD"})
        
        for elem in soup.select('article div div div div a'):
            # 如果新獲得的貼文連結不在列表裡，則加入
            if elem['href'] not in post_url:
                post_url.append(elem['href'])
        
        if img == next_img:
            break
        # 尋找所有的貼文連結
    


    # 總共加載的貼文連結數
    print(tg_lst[case] + ":總共取得 " + str(len(post_url)) + " 篇貼文連結")


    # ----- 獲取貼文讚數、留言數 ---
    browser.get(url+tg_lst[case])
    post_info = []

    for i in range(len(post_url)):
        find = False
        post_one = []
        while not find:
            try:
                # 找到對應的貼文，鼠標移入
                post_elem = browser.find_element_by_xpath('//a[@href="'+str(post_url[i])+'"]')
                action = ActionChains(browser)
                action.move_to_element(post_elem).perform()
                n_like_elem = browser.find_elements_by_class_name('-V_eO')
                for j in n_like_elem:
                    post_one.append(j.text)
                post_info.append(post_one)
                find = True
            except:
                # 找不到就往下滑
                scroll = 'window.scrollBy(0,250)'
                browser.execute_script(scroll)
                time.sleep(randint(100,150)/100)
                continue

    # ---- 擷取貼文內容 ----
    content_info = []
    for i in range(len(post_url)):
        try:
            if i % 10 == 0 and i != 0:
                browser.get(url+tg_lst[randint(0,len(tg_lst))])
            browser.get(url+post_url[i])
            time.sleep(randint(100,200)/100)
            #WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span')))  
            content_info.append(browser.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span').text)
            time.sleep(randint(50,100)/100)
        except:
            content_info.append('')
            time.sleep(randint(50,100)/100)
            continue


    info = []
    info.append(num_info)
    info.append(intro_info)
    info.append(post_info)
    info.append(content_info)
    result.append(info)

# 開啟輸出的 CSV 檔案
with open('output.csv', 'w', newline='',encoding="utf-8") as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)
    # 寫入一列資料
    
    for i in range(len(result)):
        writer.writerow([tg_lst[i]])
        for j in range(len(result[i])):
            writer.writerow(result[i][j])

daifuku_channel:總共取得0篇貼文連結
leila.love8764:總共取得 216 篇貼文連結
maokeli01:總共取得 302 篇貼文連結
kk__1491:總共取得 84 篇貼文連結
12___yu___29:總共取得 33 篇貼文連結
trend_musically:總共取得0篇貼文連結
draw.1989:總共取得 25 篇貼文連結
_chih.wei_:總共取得 3 篇貼文連結
liu___023:總共取得 45 篇貼文連結
eatout_tw:總共取得 194 篇貼文連結
nutruelife:總共取得 363 篇貼文連結


WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.72)


In [153]:
posi = []
for i in range(14):
    try:
        if len(result[i][2]) == int(result[i][0][0]):
            posi.append(i)
    except:
        posi.append(i)

In [154]:
output = []
for i in range(14):
    if i in posi:
        temp=[]
        temp.append([tg_lst[i]])
        for j in range(4):
            temp.append(result[i][j])
        output.append(temp)

In [155]:
import csv

# 開啟輸出的 CSV 檔案
with open('output1.csv', 'w', newline='',encoding="utf-8") as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)
    # 寫入一列資料
    
    for i in range(len(output)):
        for j in range(len(output[i])):
            writer.writerow(output[i][j])

In [1]:
import csv

r = []
with open('output.csv', newline='',encoding="utf-8") as csvfile:
    # 以冒號分隔欄位，讀取檔案內容
    rows = csv.reader(csvfile, delimiter=',')
    for j in rows:
        r.append(j)

a = []
for i in range(int(len(r)/5)):
    b = []
    for j in range(5):
        b.append(r[i*5+j])
    a.append(b)

In [2]:
a

[[['ray_easylife'], ['NaN', '89', '92'], ['顏宏宇 Ray'], [''], ['']],
 [['lyndeb___1204'],
  ['16', '202', '643'],
  ['🇹🇼'],
  ["['15', 'NaN']",
   "['14', 'NaN']",
   "['20', 'NaN']",
   "['18', 'NaN']",
   "['15', '2']",
   "['23', 'NaN']",
   "['18', 'NaN']",
   "['19', 'NaN']",
   "['18', 'NaN']",
   "['20', 'NaN']",
   "['19', '2']",
   "['20', 'NaN']",
   "['23', 'NaN']",
   "['19', 'NaN']",
   "['13', 'NaN']",
   "['12', 'NaN']"],
  ['🎂\U0001f972',
   '🌝🥮💗',
   '剩這張，好像來不及穿學士服拍了\n👵🏻👧🏻#阿嬤',
   '#vincentvangogh',
   'You got me feeling like a psycho, psycho🌚 #vincentvangogh',
   '✨✨✨✨✨\n#2019 #morgandooley #englishdramacontest',
   '#煙火 #🎆 #大稻埕碼頭',
   '🏠',
   '🐮🌻🌞🍼🍧🍡\n#summeriscoming\n#happysummervacation',
   '👨\u200d👩\u200d👧\u200d👦👫👭',
   '冬天就是要吃冰🤪',
   '1️⃣✨20y✨🌧🌬☔️🌊🌫',
   '薯條🍟好吃\n#bff',
   '🌸',
   '👀#ghibli',
   '🌕🌕🌕🌕🌗']],
 [['warriors6210'],
  ['157', '158', '169'],
  ['Angel & Rika\n經常在半夜想吃炸雞的女子，即使白天已經吃過。\nheatheryyyy 和 adleryang 都在追蹤'],
  ["['55', 'NaN']",
   "['12', 'NaN']",
 

In [35]:
:
    
    a=[]
for i in range(len(post_url)):
    b = []
    try:
        b.append(int(info[2][i][0]))
        b.append(int(info[2][i][1]))
    except:
        print("數值為Na")
        b.append(None)
        continue
    a.append(b)
a

數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na
數值為Na


[[38, 6],
 [37, 6],
 [39, 2],
 [47, 2],
 [30, 9],
 [36, 2],
 [92, 1],
 [50, 1],
 [40, 2],
 [31, 1]]